In [1]:
import os, sys, glob, json, scipy as sp
import matplotlib.pyplot as plt
import itertools
os.chdir("../")
path = os.getcwd()
sys.path.insert(0, path)
import GWGen
from GWGen.DressedFluxes import *
import numpy as np
from numpy import array
import scipy as sp
import time
import scipy.optimize as spopt
import scipy.interpolate as spint
import glob
import matplotlib.pyplot as plt
import re
alpha_match_1 = re.compile("Alpha_\d+_\d+")
alpha_match_2 = re.compile("Alpha_\d+")
modeovertonematch = re.compile("Mode_1_Overtone_0")
alpha_rege = re.compile("\d+")
def cartesian_product(*arrays):
    la = len(arrays)
    dtype = np.result_type(*arrays)
    arr = np.empty([len(a) for a in arrays] + [la], dtype=dtype)
    for i, a in enumerate(np.ix_(*arrays)):
        arr[...,i] = a
    return arr.reshape(-1, la)

In [2]:
allfilenames=glob.glob(pathToSolutionSet+"BHSpin*")
alldata = [np.load(i) for i in allfilenames]
modeovertonebool  = [bool(re.search("Mode_1_Overtone_0",i.fid.name)) for i in alldata]
newalldata =[]
for inx,boolval in enumerate(modeovertonebool):
    if boolval:
        newalldata.append(alldata[inx])
alldata = newalldata

In [ ]:
interpdat = []
for inx,dat in enumerate(alldata):
    raddat = dat["RadialData"]
    thetdat = dat["ThetaData"][0:100]
    nonnanthetdat = notnan(thetdat)
    if len(nonnanthetdat)>100:
        nonnanthetdat[0:100]
    endat = dat["EnergyData"][:,nonnanthetdat]
    interpfunc = spint.RectBivariateSpline(raddat,thetdat[nonnanthetdat],endat)
    alphastr = alpha_match_1.findall(dat.fid.name)
    if len(alphastr)==0:
        alphastr=alpha_match_2.findall(dat.fid.name)
    if isinstance(alphastr, str):
        pass
    elif isinstance(alphastr,list):
        alphastr=alphastr[0]
    alphapieces = alpha_rege.findall(alphastr)
    if len(alphapieces)>1:
        alphavalue = float(alphapieces[0])/float(alphapieces[1])
    else:
        alphavalue = float(alphapieces[0])
    out = [alphavalue,min(raddat), max(raddat), min(thetdat), max(thetdat), interpfunc]
    interpdat.append(out)

In [ ]:
alphavals = []
maxradial = []
maxendens = []
interpsfuncs = []
for inx, interptuple in enumerate(interpdat):
    alpha, minr,maxr,mint,maxt,interpfunc = interptuple
    rad = 50
    thet = np.pi/2
    interpdata = interpfunc(rad,thet)
    maxenden = max(interpdata)
    maxradial.append(maxr)
    alphavals.append(alpha)
    maxendens.append(maxenden)
    interpsfuncs.append(interpfunc)

In [ ]:
indexsort = np.argsort(alphavals)
sorted_alphavals = np.asarray(alphavals)[indexsort]
sorted_interpfuncs = np.asarray(interpsfuncs)[indexsort]
sorted_maxradial = np.asarray(maxradial)[indexsort]

In [ ]:
takenumber = 2
take_alphavalues = sorted_alphavals[:takenumber]
minmaxrad = min(sorted_maxradial[:takenumber])
maxmaxrad = max(sorted_maxradial[:takenumber])
radialdom = np.arange(minr, minmaxrad,1)
take_interpfuncs = sorted_interpfuncs[:takenumber]

In [ ]:
energydata

In [ ]:
aa =time.time()
alpharange = take_alphavalues
radialrange = radialdom[:int(len(radialdom)/3)]
thetarange = thetdat
energydata = np.asarray([i(radialrange,thetarange) for i in take_interpfuncs])
RGI = spint.RegularGridInterpolator((alpharange,radialrange,thetarange), energydata, fill_value=None, bounds_error=False, method="linear")
bb=time.time()
print(bb-aa)
coords = cartesian_product(np.array([0.065]), radialrange, thetarange)
dd=time.time(); print("coords: ", dd-bb)
RGIdata = np.reshape(RGI(coords), ( len(radialrange), len(thetarange)))
cc=time.time(); print("RGI data: ",cc-dd)
RBI = spint.RectBivariateSpline(radialrange, thetarange, RGIdata)
print(time.time()-cc)

In [ ]:
tmpRBIdata = RBI(radialrange, thetarange)[:,50]
tmpRGIdata = RGIdata[:,50]
plt.plot(tmpRBIdata)
plt.plot(tmpRGIdata)
plt.show();
plt.plot(tmpRBIdata-tmpRGIdata);

In [ ]:
coords = np.asarray([[alpha, r, theta] for alpha in alpharange for r in radialdom for theta in thetarange])

In [ ]:
energydatain = np.ravel(energydata)
rbfint = spint.RBFInterpolator(coords,energydatain, neighbors=800,kernel="linear",smoothing=0,degree=0)

In [ ]:
TMPenden=array([[[0.00033903, 0.00034132, 0.00034312, 0.00034442, 0.0003452 ,
         0.00034547, 0.0003452 , 0.00034441, 0.00034311, 0.0003413 ],
        [0.00041104, 0.0004126 , 0.00041383, 0.00041471, 0.00041524,
         0.00041542, 0.00041524, 0.00041471, 0.00041382, 0.00041259],
        [0.00044731, 0.00044845, 0.00044934, 0.00044999, 0.00045037,
         0.0004505 , 0.00045037, 0.00044998, 0.00044934, 0.00044844],
        [0.00046978, 0.00047068, 0.00047138, 0.00047189, 0.00047219,
         0.00047229, 0.00047219, 0.00047188, 0.00047138, 0.00047067],
        [0.00048431, 0.00048505, 0.00048563, 0.00048604, 0.00048629,
         0.00048638, 0.00048629, 0.00048604, 0.00048563, 0.00048504],
        [0.00049386, 0.00049449, 0.00049499, 0.00049534, 0.00049555,
         0.00049562, 0.00049555, 0.00049534, 0.00049498, 0.00049449],
        [0.00050013, 0.00050068, 0.00050111, 0.00050142, 0.0005016 ,
         0.00050166, 0.0005016 , 0.00050142, 0.00050111, 0.00050068],
        [0.00050414, 0.00050462, 0.000505  , 0.00050527, 0.00050544,
         0.00050549, 0.00050544, 0.00050527, 0.000505  , 0.00050462],
        [0.00050653, 0.00050696, 0.0005073 , 0.00050754, 0.00050769,
         0.00050774, 0.00050769, 0.00050754, 0.0005073 , 0.00050696],
        [0.00050773, 0.00050812, 0.00050843, 0.00050865, 0.00050878,
         0.00050882, 0.00050878, 0.00050865, 0.00050843, 0.00050812]],

       [[0.00047842, 0.00048166, 0.00048422, 0.00048606, 0.00048717,
         0.00048755, 0.00048717, 0.00048605, 0.0004842 , 0.00048164],
        [0.00058177, 0.00058398, 0.00058571, 0.00058696, 0.00058771,
         0.00058796, 0.00058771, 0.00058695, 0.0005857 , 0.00058396],
        [0.00063274, 0.00063436, 0.00063562, 0.00063653, 0.00063708,
         0.00063726, 0.00063708, 0.00063653, 0.00063562, 0.00063435],
        [0.00066297, 0.00066424, 0.00066524, 0.00066595, 0.00066638,
         0.00066653, 0.00066638, 0.00066595, 0.00066523, 0.00066424],
        [0.00068146, 0.00068251, 0.00068333, 0.00068392, 0.00068427,
         0.00068439, 0.00068427, 0.00068392, 0.00068333, 0.00068251],
        [0.00069271, 0.00069361, 0.0006943 , 0.0006948 , 0.0006951 ,
         0.0006952 , 0.0006951 , 0.0006948 , 0.0006943 , 0.0006936 ],
        [0.00069925, 0.00070002, 0.00070062, 0.00070106, 0.00070132,
         0.00070141, 0.00070132, 0.00070106, 0.00070062, 0.00070001],
        [0.00070256, 0.00070325, 0.00070378, 0.00070416, 0.00070439,
         0.00070447, 0.00070439, 0.00070416, 0.00070378, 0.00070324],
        [0.00070361, 0.00070422, 0.0007047 , 0.00070505, 0.00070525,
         0.00070532, 0.00070525, 0.00070504, 0.0007047 , 0.00070422],
        [0.00070302, 0.00070358, 0.00070401, 0.00070432, 0.0007045 ,
         0.00070457, 0.0007045 , 0.00070432, 0.00070401, 0.00070357]],

       [[0.00064677, 0.00065116, 0.00065462, 0.00065712, 0.00065863,
         0.00065914, 0.00065863, 0.00065711, 0.00065461, 0.00065114],
        [0.00078754, 0.00079052, 0.00079286, 0.00079455, 0.00079556,
         0.0007959 , 0.00079556, 0.00079454, 0.00079285, 0.0007905 ],
        [0.00085515, 0.00085733, 0.00085904, 0.00086027, 0.00086101,
         0.00086126, 0.00086101, 0.00086026, 0.00085903, 0.00085732],
        [0.00089318, 0.0008949 , 0.00089624, 0.0008972 , 0.00089778,
         0.00089798, 0.00089778, 0.0008972 , 0.00089623, 0.00089489],
        [0.00091474, 0.00091616, 0.00091726, 0.00091805, 0.00091853,
         0.00091869, 0.00091853, 0.00091805, 0.00091726, 0.00091615],
        [0.00092632, 0.00092752, 0.00092845, 0.00092913, 0.00092953,
         0.00092967, 0.00092953, 0.00092912, 0.00092845, 0.00092751],
        [0.00093148, 0.00093252, 0.00093333, 0.00093391, 0.00093426,
         0.00093438, 0.00093426, 0.00093391, 0.00093332, 0.00093251],
        [0.00093233, 0.00093325, 0.00093396, 0.00093448, 0.00093479,
         0.00093489, 0.00093478, 0.00093447, 0.00093396, 0.00093324],
        [0.0009302 , 0.00093102, 0.00093166, 0.00093212, 0.00093239,
         0.00093248, 0.00093239, 0.00093211, 0.00093166, 0.00093102],
        [0.00092595, 0.00092669, 0.00092726, 0.00092768, 0.00092792,
         0.00092801, 0.00092792, 0.00092767, 0.00092726, 0.00092668]]])
TMPraddata=array([ 2.43597707,  3.43597707,  4.43597707,  5.43597707,  6.43597707,
        7.43597707,  8.43597707,  9.43597707, 10.43597707, 11.43597707])
TMPthetadata=array([1.41381669, 1.44523262, 1.47664855, 1.50806447, 1.5394804 ,
       1.57089633, 1.60231225, 1.63372818, 1.66514411, 1.69656003])
TMPalpha = np.array([0.06,0.07,0.08])

coords = np.asarray([[alpha,r,theta] for alpha in TMPalpha for r in TMPraddata for theta in TMPthetadata])
yvalues = np.ravel(TMPenden)
tmprbf = spint.RBFInterpolator(
    coords,
    yvalues,
    neighbors=1000,
    kernel='linear',
    smoothing=0,
    degree=2
)

tmpalpha = 0.6
pnts = [[tmpalpha,i,TMPthetadata[5]] for i in TMPraddata]
datpnts = tmprbf(pnts)
plt.scatter(TMPraddata, datpnts, label="Radial Basis Function",marker=".", linewidth=.5)
plt.plot(TMPraddata, TMPenden[0,:,5],label="Data")
plt.legend();

In [ ]:
aa=time.time()
thetaval = np.pi/2
extendedradialdom = np.tile(radialdom,takenumber)
extendedalphavals = np.repeat(take_alphavalues,len(extendedradialdom)/takenumber)
extendedenergy2d = [func(radialdom,thetaval) for func in take_interpfuncs]
extendedenergy = np.ravel(extendedenergy2d)
print(time.time()-aa)

In [ ]:
fig = plt.figure(figsize=(16,8))
ax = fig.add_subplot(projection='3d')
ax.scatter(extendedalphavals,extendedradialdom, extendedenergy);

In [ ]:
neighbordata = ProcaSolution._ProcaSolution__get_closest_datasets(1,0.0703415,0.85)
alphane = neighbordata["alphaneighbors"]
spinne = neighbordata["bhspinneighbors"]
datasets = neighbordata["datasets"]

radialdata = [i["RadialData"] for i in datasets]
thetadata = [i["ThetaData"] for i in datasets]
newshape = (int(min([len(i) for i in radialdata])/7),100)
radialdata = [i[:newshape[0]] for i in radialdata]
thetadata = [i[:newshape[1]] for i in thetadata]

rawen = [i["EnergyData"][:newshape[0], :newshape[1]] for i in datasets]
rawenoldshape = np.shape(rawen)
#first index denotes bh spin value
#second index denotes alpha value
#third denotes radial value
#fourth denotes theta value
rawenreshaped = np.reshape(rawen, (2,2,rawenoldshape[1],rawenoldshape[2]))

if not len(np.unique(spinne))==len(spinne) and not len(np.unique(alphane))==len(alphane):
    interp = spint.RegularGridInterpolator((radialdata[0], thetadata[0]), rawenreshaped[0][0])
    coords = cartesian_product(np.array(radialdata[0]), np.array(thetadata[0]))
if not len(np.unique(spinne))==len(spinne) and len(np.unique(alphane))==len(alphane):
    interp = spint.RegularGridInterpolator((alphane, radialdata[0], thetadata[0]), rawenreshaped[0])
    coords = cartesian_product(np.array([0.07]), np.array(radialdata[0]), np.array(thetadata[0]))

if len(np.unique(spinne))==len(spinne) and not len(np.unique(alphane))==len(alphane):
    interp = spint.RegularGridInterpolator((spinne, radialdata[0], thetadata[0]), rawenreshaped[:,0])
    coords = cartesian_product(np.array([0.85]),np.array(radialdata[0]), np.array(thetadata[0]))

if len(np.unique(spinne))==len(spinne) and len(np.unique(alphane))==len(alphane):
    interp = spint.RegularGridInterpolator((spinne, alphane, radialdata[0], thetadata[0]), rawenreshaped)
    coords = cartesian_product(np.array([0.85]), np.array([0.07]),np.array(radialdata[0]), np.array(thetadata[0]))

values = np.reshape(interp(coords), newshape)
interp = spint.RectBivariateSpline(radialdata[0], thetadata[0], values)

## Sorting Proca Data

First sort by BHspin values, then group by alpha values and take neighboring data

In [125]:
test_bhspin = 0.79
test_alpha = 0.135

In [133]:
aa=time.time()
bhspins = list(map(BHSpinValFromFilename,allfilenames))
unsorted_alphavalues = list(map(AlphaValFromFilename, allfilenames))
index_sort = np.argsort(bhspins)
bhspins = np.array(bhspins)[index_sort]
allfilenames = np.array(allfilenames)[index_sort]
res=itertools.groupby(allfilenames, BHSpinValFromFilename)
sorted_filenames={}
for key,group in res:
    grouplist = list(group)
    alphas = list(map(AlphaValFromFilename, grouplist))
    index_sort = np.argsort(alphas)
    sorted_group = np.array(grouplist)[index_sort]
    sorted_filenames[key] = sorted_group



bhspins = list(sorted_filenames.keys())
rindex = bisect_right(bhspins,test_bhspin)
Larger_BHSpin_Datasets = sorted_filenames[bhspins[rindex]]
Smaller_BHSpin_Datasets = sorted_filenames[bhspins[rindex-1]]

Smaller_BHSpin_AlphaValues = list(map(AlphaValFromFilename, Smaller_BHSpin_Datasets))
Larger_BHSpin_AlphaValues = list(map(AlphaValFromFilename, Larger_BHSpin_Datasets))
assert test_alpha>Smaller_BHSpin_AlphaValues[0] and test_alpha<Smaller_BHSpin_AlphaValues[-1] and test_alpha>Larger_BHSpin_AlphaValues[0] and test_alpha<Larger_BHSpin_AlphaValues[-1], "Error: Alpha value outside range of available data"

rindex_smaller = bisect_right(Smaller_BHSpin_AlphaValues, test_alpha)
rindex_larger = bisect_right(Larger_BHSpin_AlphaValues, test_alpha)


SmallerSmaller_Dataset = Smaller_BHSpin_Datasets[rindex_smaller-1]
SmallerLarger_Dataset = Smaller_BHSpin_Datasets[rindex_smaller]
LargerSmaller_Dataset = Larger_BHSpin_Datasets[rindex_larger-1]
LargerLarger_Dataset = Larger_BHSpin_Datasets[rindex_larger]




time.time()-aa

0.005158662796020508

In [127]:
print(SmallerSmaller_Dataset)
print(SmallerLarger_Dataset)
print(LargerSmaller_Dataset)
print(LargerLarger_Dataset)

/mnt/Data_Volume/Documents/Github/GWGenerator/GWGen/DressedFluxes/../ProcaData/BHSpin_39_50_Alpha_3_25_Mode_1_Overtone_0.npz
/mnt/Data_Volume/Documents/Github/GWGenerator/GWGen/DressedFluxes/../ProcaData/BHSpin_39_50_Alpha_3_20_Mode_1_Overtone_0.npz
/mnt/Data_Volume/Documents/Github/GWGenerator/GWGen/DressedFluxes/../ProcaData/BHSpin_21_25_Alpha_13_100_Mode_1_Overtone_0.npz
/mnt/Data_Volume/Documents/Github/GWGenerator/GWGen/DressedFluxes/../ProcaData/BHSpin_21_25_Alpha_7_50_Mode_1_Overtone_0.npz


In [129]:
print(test_bhspin)
print(test_alpha)

0.79
0.135


In [130]:
39/50

0.78

In [132]:
bhspins

[0.6, 0.62, 0.64, 0.66, 0.68, 0.72, 0.74, 0.76, 0.78, 0.84, 0.88]

# Check output of GWGen.DressedFluxes.Fluxes.ProcaSolution

In [3]:
pc = ProcaSolution(1e6,0.879, 1e-17)

UltralightBoson instance not provided. Instantiating class...
done.
